# Wide and CNN
Google 에서 발표현 Wide and Deep 을 약간 변형한 형태로 Wide(Logistic)과 Cnn(Deep)을 Combine 형태의 Network 를 개발하였다. <br> 
여기서는 자연어를 Wide and Cnn 을 사용하여 Classification 하는 것을 테스트 하고자 한다. 

In [73]:
import requests
import json, os

nn_id = 'nn902995'  # put some key value you want to test

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
print("done")

done


# 네트워크 및 그래프 버전 생성 


In [74]:
####(1) 네트워크 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

####(2) 버전 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : nn902995
evaluation result : nn902995


# 그래프 플로우 정의

In [75]:
# Work Flow 틀을 구성하도로고 지시한다. (정해진 틀을 강제로 생성)
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/' + nn_id +'/wfver/1/',
                     json={
                         "type": 'wcnn'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : wcnn


# 데이터 업로드

In [76]:
import os

return_dict = {}
return_dict['test'] = open('../../data/seq2seq_several_case.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']


# 데이터 노드 정의

In [77]:
# (1) Train Data Node 의 속성을 정의
# 어디서 Source 를 어떻게 가지고 올것인지 파라메터 정의 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })

# 전처리는 어떤 것을 할지 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/data_node/',
                      json={
                          "preprocess":  "none",
                      })
# 전처리가 완료된 데이터는 어디에 저장을 할지 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/data_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/nn902995/1/data_node


# 데이터 Feeder 정의

In [78]:
# (2) Network 에 데이터를 Feed하는 Node 의 속성을 정의 
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/wcnn/nnid/'+nn_id+'/ver/1/node/pre_feed_train/',
                     json={
                         "encode_column" : "encode",
                         "decode_column" : "decode",
                         "channel" : 1,
                         "encode_len" : 15,
                         "preprocess": "mecab",
                         "vocab_size" : 100,
                         "lable_size" : 10, 
                         "embed_type" : 'onehot'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'encode_len': 15, 'embed_type': 'onehot', 'decode_column': 'decode', 'preprocess': 'mecab', 'lable_size': 10, 'channel': 1, 'vocab_size': 100, 'encode_column': 'encode'}


# 네트워크 정의

In [79]:
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/wcnn/nnid/'+nn_id+'/ver/1/node/netconf_node/',
                     json={
                         "param":{"epoch": 1
                                  ,"traincnt": 1
                                  ,"batch_size":25000
                                  ,"predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                    "layeroutputs":32,
                                    "net_type":"cnn",
                                    "eval_type":"category",
                                    "optimizer":"AdamOptimizer" #RMSPropOptimizer
                                     }
                         ,"layers": [
                                    {"active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    },
                                    {"active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [1, 1],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                                   ]
                         ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
                         ,"labels":[]
                        })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : {'model_path': '/hoya_model_root/nn902995/1/netconf_node', 'out': {'padding': 'SAME', 'node_out': 625, 'active': 'softmax'}, 'layers': [{'layercnt': 2, 'maxpoolstride': [2, 2], 'padding': 'SAME', 'cnnfilter': [3, 3], 'cnnstride': [1, 1], 'droprate': '0.8', 'active': 'relu', 'maxpoolmatrix': [2, 2]}, {'layercnt': 1, 'maxpoolstride': [2, 2], 'padding': 'SAME', 'cnnfilter': [3, 3], 'cnnstride': [1, 1], 'droprate': '0.8', 'active': 'relu', 'maxpoolmatrix': [1, 1]}], 'config': {'net_type': 'cnn', 'layeroutputs': 32, 'num_classes': 10, 'eval_type': 'category', 'optimizer': 'AdamOptimizer', 'learnrate': 0.001}, 'labels': [], 'param': {'traincnt': 1, 'epoch': 1, 'predictcnt': 10, 'batch_size': 25000}}


# 테스트 프로세스 정의

In [83]:
# upload files for test
return_dict = {}
return_dict['test'] = open('../../data/seq2seq_several_case.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 소스 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 전처리 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                      json={
                          "preprocess":  "none",
                      })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 저장 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/test_data_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


# Feeder 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/wcnn/nnid/'+nn_id+'/ver/1/node/pre_feed_test/',
                     json={
                         "encode_column" : "encode",
                         "decode_column" : "decode",
                         "channel" : 1,
                         "encode_len" : 15,
                         "preprocess": "mecab",
                         "vocab_size" : 100,
                         "embed_type" : 'onehot',
                         "lable_size" : 10
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

node_name = 'eval_node'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/eval/nnid/'+nn_id+'/ver/1/node/'+node_name+'/',
                    json={
                        "type": "category",
                    })

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']
evaluation result : {'source_sql': 'all', 'store_path': '/hoya_str_root/nn902995/1/test_data_node', 'source_type': 'local', 'source_server': 'local', 'max_sentence_len': 0, 'preprocess': 'none', 'source_parse_type': 'raw', 'multi_node_flag': None, 'source_path': '/hoya_src_root/nn902995/1/test_data_node', 'type': 'csv'}
evaluation result : none
evaluation result : /hoya_str_root/nn902995/1/test_data_node
evaluation result : {'encode_len': 15, 'embed_type': 'onehot', 'decode_column': 'decode', 'preprocess': 'mecab', 'lable_size': 10, 'channel': 1, 'vocab_size': 100, 'encode_column': 'encode'}
evaluation result : {'type': 'category'}


# Train

In [161]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/1/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [None, None, None, None, '', {'labels': ['@', '#', 'UNKNOWN', 2, 3, 4, 6, 7, -1], 'predicts': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 46, 0, 0, 0, 0, 0], [0, 0, 0, 0, 18, 0, 0, 0, 0], [0, 0, 0, 0, 0, 36, 0, 0, 0], [0, 0, 0, 0, 0, 0, 26, 0, 0], [0, 0, 0, 0, 0, 0, 0, 16, 0], [0, 0, 0, 0, 0, 0, 0, 0, 8]]}]


# Predict

In [162]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[회의] 를 [이름] [이름] 참석자로 예약해줘" }
                     )
data = json.loads(resp.json())
print("evaluation result(7) : {0}".format(data)) 

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))